# Extractor Tutorial 

## Table of Contents

- [Real Data](#Real-Data)
- [Display area of interest](#Display-area-of-interest)
- [Create a UGrid](#Create-a-UGrid)
- [Create an Extractor](#Create-an-Extractor)
- [Setup Polyline](#Setup-Polyline)
- [Set the point scalars for the water depth](#Set-the-point-scalars-for-the-water-depth)
- [Extract the data](#Extract-the-data)
- [View the Data](#View-the-Data)

## Real Data 
The purpose of this tutorial is to show how to use the classes defined in xmsextractor to extract data at given locations on an XmUGrid with scalar values, using outputs from a numerical model.

First, we will load a UGrid containing elevation values.  The data comes from a Sedimentation and River Hydraulics – Two-Dimensional (SRH-2D) model.

In [1]:
import xms.extractor as xmsextractor
import xms.grid as xmsgrid

The following three code cells read in the data for our UGrid.

In [2]:
# A list that will hold the locations of our UGrid (x,y,z) coordinates
ugrid_points = []

# Read in our locations from a file
with open('PointsData.txt','r') as f:
    # Each line has an individual location
    for line in f:
        # Extract our location
        line = line.strip()
        line = line.strip('[,()]')
        line = line.split(',')
        point = (float(line[0]), float(line[1]), float(line[2]))
        ugrid_points.append(point)

In [3]:
# A list that will hold the cell data of our UGrid
ugrid_cells = []

# Read in the cell data from a file
with open('CellsData.txt','r') as f:
    # Each line has the data for an individual cell
    for line in f:
        # Extract our cell data
        line = line.strip()
        line = line.rstrip(',')
        line = line.split(',')
        ugrid_cells.extend([xmsgrid.ugrid.UGrid.cell_type_enum.TRIANGLE, int(line[1]), int(line[2]), int(line[3]), int(line[4])])

In [4]:
# A list that will hold the depth data for each location on our UGrid
depths = []

# Read in the depth data from a file
with open('DepthsData.txt','r') as f:
    # Each line has the depth for an individual location
    for line in f:
        # Extract our depth
        line = line.strip()
        line = line.rstrip(',')
        depths.append(float(line))

## Display area of interest

The code below uses `cartopy.crs` and `geoviews` to render our area of interest. An image of this render is shown below.

```python
srs = ccrs.LambertConformal(central_longitude=-98, central_latitude=35.0, false_easting=600000,
                           standard_parallels=(35.5667, 36.7667))
boundary_polygon = gv.Shape.from_shapefile("boundary_polys.shp", crs=srs).options(color="red", alpha=0.25)
basemap = gv.tile_sources.ESRI()

%opts Shape Polygons Points [height=700 width=700]
basemap * boundary_polygon
```

![Area of Interest](images/area_of_interest.png)

## Create a UGrid

In [5]:
ugrid = xmsgrid.ugrid.UGrid(points=ugrid_points, cellstream=ugrid_cells)

##### View the UGrid

The code below uses `geoviews` to render our ugrid with our area of interest. An image of this render is shown below.

```python
ugrid_cells = []

for i in range(0, ugrid.get_cell_count()):
    ugrid_cells.extend(ugrid.get_cell_cellstream(i)[1][2:])

polygons, lines, points = ugrid_tools.create_hv_objects_for_image(ugrid, _projection=srs)
basemap * polygons * lines * points
```

![UGrid](images/ugrid.png)

## Create an Extractor

In [6]:
extractor = xmsextractor.UGrid2dPolylineDataExtractor(ugrid, 'points')

## Setup Polyline

In [7]:
polyline = [(657675.3, 110106.7, 275.0), (657716.7, 110192.5, 275.0)]
extractor.set_polyline(polyline)

In [8]:
extracted_locations = extractor.extract_locations

## Set the point scalars for the water depth

In [9]:
extractor.set_grid_scalars(depths, [], 'cells')
print(ugrid.locations)

[[6.58306098e+05 1.09672020e+05 2.75000000e+02]
 [6.58307941e+05 1.09679038e+05 2.75000000e+02]
 [6.58323570e+05 1.09697156e+05 2.75000000e+02]
 ...
 [6.58940545e+05 1.10397323e+05 2.70000000e+02]
 [6.58927408e+05 1.10385861e+05 2.70000000e+02]
 [6.58914221e+05 1.10373607e+05 2.70000000e+02]]


## Extract the data

In [10]:
extracted_data = extractor.extract_data()
print(str(["{0:0.4f}".format(i) for i in extracted_data]).replace("'", ""))

[0.4364, 0.4364, 0.7657, 0.9219, 1.3677, 1.5640, 1.7878, 1.8955, 1.9747, 2.0098, 2.0801, 2.1397, 2.1662, 2.2067, 2.2315, 2.2453, 2.2668, 2.2565, 2.2500, 2.2132, 2.2099, 2.1895, 1.8264, 1.4297, 1.4297]


### View the Data

The code below uses `holoviews` to create a render that visualizes our extracted data. An image of this render is shown below.

```python
pxn = extracted_locations[-1][0]
px0 = extracted_locations[0][0]
distance = pxn - px0
hv_point_locations = [(extracted_locations[i][0] - px0, extracted_data[i]) for i in range(0, len(extracted_data))]
height = max([y[1] for y in hv_point_locations])

x_range = (min([x[0] for x in hv_point_locations]) - (distance*0.1), max([x[0] for x in hv_point_locations]) + (distance*0.1))
y_range = (min([y[1] for y in hv_point_locations]) - (height*0.1),   max([y[1] for y in hv_point_locations]) + (height*0.1))

%opts Points [colorbar=True tools=['hover']] (size=4 color='red')
graph_points = hv.Points(hv_point_locations)
graph_line = hv.Path([hv_point_locations,],)
graph_line * graph_points.redim.range(x=x_range, y=y_range)
```

![Data](images/data.png)